In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("tree").getOrCreate()

In [3]:
data = spark.read.csv('College.csv',inferSchema=True, header=True)

In [4]:
data.describe().show()

+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
|summary|              School|Private|              Apps|            Accept|          Enroll|         Top10perc|         Top25perc|      F_Undergrad|      P_Undergrad|          Outstate|        Room_Board|             Books|          Personal|               PhD|          Terminal|         S_F_Ratio|       perc_alumni|          Expend|         Grad_Rate|
+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------

In [5]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [9]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')

In [10]:
output = assembler.transform(data)

In [11]:
from pyspark.ml.feature import StringIndexer

In [12]:
indexer = StringIndexer(inputCol='Private',outputCol='PrivateIndex')

In [13]:
output_fixed = indexer.fit(output).transform(output)

In [14]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [15]:
final_data = output_fixed.select('features','PrivateIndex')

In [16]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [19]:
from pyspark.ml.classification import (DecisionTreeClassifier, GBTClassifier,
                                       RandomForestClassifier)

In [20]:
from pyspark.ml import Pipeline

In [21]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')

In [22]:
rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')

In [44]:
gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

In [45]:
dtc_model = dtc.fit(train_data)

In [46]:
rfc_model = rfc.fit(train_data)

In [48]:
gbt_model = gbt.fit(train_data)

In [49]:
dtc_preds = dtc_model.transform(test_data)

In [50]:
rfc_preds = rfc_model.transform(test_data)

In [51]:
gbt_preds = gbt_model.transform(test_data)

In [52]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [53]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [54]:
print('DTC')
print(my_binary_eval.evaluate(dtc_preds))

DTC
0.9533157930243541


In [55]:
print('RFC')
print(my_binary_eval.evaluate(rfc_preds))

RFC
0.9878567091681846


In [56]:
gbt_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [57]:
rfc_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [62]:
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex',rawPredictionCol="prediction")

In [63]:
print("GBT")
print(my_binary_eval2.evaluate(gbt_preds))

GBT
0.8814005262092693


In [64]:
rfc_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [65]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [66]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex',metricName='accuracy')

In [67]:
rfc_acc = acc_eval.evaluate(rfc_preds)

In [68]:
rfc_acc

0.928030303030303